Уважаемый студент!


…Итак, данные собраны и можно начинать анализ. Коллеги посоветовали вам выделить паттерны и на их основе построить торговую стратегию.


Вы решили посмотреть сначала самые простые паттерны на основе скользящих средних и затем дополнить их более сложными шаблонами, такими как паттерны разворота, продолжения и свечного анализа.


Также вы понимаете, что нужен инструмент оценки эффективности и поскольку, у вас уже есть данные прошлых периодов, вы решаете создать инструмент оценки эффективности новой стратегии, пока у вас есть немного свободного времени.

На основании вышесказанного, вам необходимо построить модель, которая будет принимать решение о торговых операциях по одной или группе бумаг и криптовалют, и оценить ее эффективность на отложенных данных.


Вам предлагается на основе представленной информации:


Создать код на Python, который разделит ваши данные на тренировочный, тестовый и валидационный наборы данных.
Построить одну или несколько моделей на основе паттернов технического анализа, которая будет принимать торговые решения по бумагам SnP500 и/или криптовалютам.
Провести подбор гиперпараметров моделей с использованием обучающей и тестовой выборок.
Провести финальное тестирование построенных торговых стратегий на валидационном наборе данных и сравнить их между собой.
Сформировать дашборд, показывающий эффективность различных стратегий во времени.